In [4]:
#pip install elasticsearch

# Módulos

In [51]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime
from datetime import timezone
import pytz

# Inputs 

In [88]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-04 23:50:00" # Meter la hora según timezone
fecha_fin_incidencia    = "2022-03-06 00:10:00" # Meter la hora según timezone

# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"

In [89]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

In [90]:
# Calculo del offset por cambio de hora

# Timestamps fechas incidencia
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia_timestamp = datetime.timestamp(fecha_inicio_incidencia)*1000

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia_timestamp = datetime.timestamp(fecha_fin_incidencia)*1000

# Timestamps de las fechas de cambio de hora

# Primer cambio de hora
fecha_cambio_hora_1 = "2022-03-27 02:00:00" 
fecha_cambio_hora_1 = datetime.strptime(fecha_cambio_hora_1, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_1_timestamp = datetime.timestamp(fecha_cambio_hora_1)*1000

# Segundo cambio de hora
fecha_cambio_hora_2 = "2022-10-30 02:00:00" 
fecha_cambio_hora_2 = datetime.strptime(fecha_cambio_hora_2, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_2_timestamp = datetime.timestamp(fecha_cambio_hora_2)*1000

# Comparamos fechas en el mismo año 
fecha_inicio_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_1 = -3600 

if ((fecha_inicio_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp)):
  offset_1 = 0

if (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp):
  offset_1 = 3600  

fecha_fin_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_2 = -3600 

if ((fecha_fin_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp)):
  offset_2 = 0

if (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp):
  offset_2 = 3600  

In [91]:
offset_1

-3600

# Nuevas Variables

In [92]:
# Transformacion de variables de entrada
# En vez de 3600 eliminar en funcion del mes (hasta marzo 27 ... -1, luego -2 y luego -1, mirar )
# Fechas
fecha_inicio_incidencia = str((int(fecha_inicio_incidencia_timestamp + offset_1*1000)))
print(fecha_inicio_incidencia)

fecha_fin_incidencia = str((int(fecha_fin_incidencia_timestamp + offset_2*1000)))
print(fecha_fin_incidencia)

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

# Cliente de ML
cliente_ml = client.ml

1646434200000
1646521800000


# Solución

In [93]:
# Paso 1. Parar y cerrar el Job
cliente_ml.close_job(job_id=jobname)

ObjectApiResponse({'closed': True})

In [94]:
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
snapshots_available
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))

In [95]:
for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

In [96]:
print(latest[0][1])

1669370255.0


In [97]:
# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # Ordenar por timestamp (primera columna)
latest1 = latest[latest[:,0].argsort()]
latest1
print(latest1[0][1])
print(latest1[1][1])
print(latest1[2][1])

1669370115.0
1669370173.0
1669370255.0


In [98]:
latest2 = np.flip(latest1, axis=0)
print(latest2[0][1])
print(latest2[1][1])
print(latest2[2][1])

1669370255.0
1669370173.0
1669370115.0


In [99]:
print(fecha_inicio_incidencia)

1646434200000


In [100]:
# Eliminar valores mayores
latest3 = latest2[latest2[:,0]<float(fecha_inicio_incidencia)]
print(latest3[0][0])
print(latest3[1][0])
print(latest3[2][0])

1646002200000.0


IndexError: ignored

In [ ]:
# Escoger el id de la reversion correcta
id_revert = latest3[0,1]

In [ ]:
id_revert = str(int(id_revert))
print(id_revert)

In [63]:
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
snapshots_available
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))

for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # Ordenar por timestamp (primera columna)
  latest1 = latest[latest[:,0].argsort()]
  latest2 = np.flip(latest1, axis=0)
  # Eliminar valores mayores
  latest3 = latest2[latest2[:,0]<float(fecha_inicio_incidencia)]
# Escoger el id de la reversion correcta
id_revert = latest3[0,1]
id_revert = str(int(id_revert))
print(id_revert)

1669370255


In [46]:
# Paso 3. Revertir el Job antes de la incidencia 
cliente_ml.revert_model_snapshot(job_id=jobname, snapshot_id=id_revert, delete_intervening_results=True)

ObjectApiResponse({'model': {'job_id': 'sabado_anomalia', 'min_version': '8.3.0', 'timestamp': 1669324386000, 'description': 'State persisted due to job close at 2022-11-24T21:13:06+0000', 'snapshot_id': '1669324386', 'snapshot_doc_count': 1, 'model_size_stats': {'job_id': 'sabado_anomalia', 'result_type': 'model_size_stats', 'model_bytes': 73020, 'peak_model_bytes': 79678, 'model_bytes_exceeded': 0, 'model_bytes_memory_limit': 11534336, 'total_by_field_count': 3, 'total_over_field_count': 0, 'total_partition_field_count': 2, 'bucket_allocation_failures_count': 0, 'memory_status': 'ok', 'assignment_memory_basis': 'current_model_bytes', 'categorized_doc_count': 0, 'total_category_count': 0, 'frequent_category_count': 0, 'rare_category_count': 0, 'dead_category_count': 0, 'failed_category_count': 0, 'categorization_status': 'ok', 'log_time': 1669324386569, 'timestamp': 1648550400000}, 'latest_record_time_stamp': 1648551000000, 'latest_result_time_stamp': 1648550400000, 'retain': False}})

In [47]:
# Paso 4. Abrir el Job 
cliente_ml.open_job(job_id=jobname)

ObjectApiResponse({'opened': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

In [48]:
# Paso 5. Avanzamos datafeed hasta la fecha de inicio de incidencia
cliente_ml.start_datafeed(datafeed_id=datafeed_id, end=fecha_inicio_incidencia)

ObjectApiResponse({'started': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

In [50]:
# Paso 6. Abrir el Job 
cliente_ml.open_job(job_id=jobname)

ConflictError: ignored

In [ ]:
# Paso 7. Avanzamos datafeed desde la fecha de fin de incidencia hasta la fecha actual
cliente_ml.start_datafeed(datafeed_id=datafeed_id, start=str(int(fecha_fin_incidencia)))

ObjectApiResponse({'started': True, 'node': '1UbOBFWrRuiIttmnruod0A'})